In [1]:
import pandas as pd
import numpy as np

from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk

In [6]:
clean_mini = pd.read_csv('clean_r_stack_060220.csv')

In [7]:
clean_mini.head()

,question,summary,tags,user_id,no_of_votes,no_of_answers,date,clean_text
0,How do I upload a R dataframe as a CSV file on...,I'm trying to convert a dataframe in R to a CS...,"['r', 'azure', 'azure-storage-blobs']",Peter Pan,2.0,1.0,2019-08-02,upload r dataframe csv file azure blob storage
1,Filtering a dataframe with respect to the list...,I am in trouble with making multi filtering in...,"['r', 'tidyverse']",Steve Lee,1.0,1.0,2019-08-02,filtering dataframe respect list datum
2,r data frame check next n number of variables,I have a data frame as follows where x is a bo...,"['r', 'dataframe']",TheN,1.0,2.0,2019-08-02,r data frame check next n number variable
3,Matching based on column vector,I need to create a matrix based on matching t...,"['r', 'matrix', 'pattern-matching', 'match']",JC3019,0.0,0.0,2019-08-02,match base column vector
4,Converting dgCMatrix to logical matrix,Consider this simple sparse matrix\n\n> (X <- ...,"['r', 'matrix', 'sparse-matrix', 'logical-oper...",Roland,3.0,2.0,2019-08-02,convert dgcmatrix logical matrix


In [8]:
clean_mini.shape

(325306, 8)

In [9]:
clean_mini['clean_text'].isna().sum()

6

In [10]:
clean_mini = clean_mini.dropna(subset=['clean_text'])

In [11]:
clean_mini['clean_text'] = clean_mini['clean_text'].str.split(' ')

In [12]:
clean_mini['clean_text'].head()

0    [upload, r, dataframe, csv, file, azure, blob,...
1         [filtering, dataframe, respect, list, datum]
2    [r, data, frame, check, next, n, number, varia...
3                        [match, base, column, vector]
4                [convert, dgcmatrix, logical, matrix]
Name: clean_text, dtype: object

In [13]:
def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, additional_stopwords=set(), no_below=5, no_above=0.5):
  print('Building dictionary...')
  dictionary = Dictionary(docs)
  stopwords = nltk_stopwords().union(additional_stopwords)
  stopword_ids = map(dictionary.token2id.get, stopwords)
  dictionary.filter_tokens(stopword_ids)
  dictionary.compactify()
  dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
  dictionary.compactify()

  print('Building corpus...')
  corpus = [dictionary.doc2bow(doc) for doc in docs]

  return dictionary, corpus

In [14]:
dictionary, corpus = prep_corpus(clean_mini['clean_text'])

Building dictionary...
Building corpus...


In [19]:
MmCorpus.serialize('models/r_corpus.mm', corpus)
dictionary.save('models/r_dict.dict')

In [20]:
%%time
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=15)
                                      
lda.save('models/r_lda.model')

CPU times: user 1min 33s, sys: 529 ms, total: 1min 33s
Wall time: 1min 34s


In [17]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [18]:
%%time
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

CPU times: user 7min 33s, sys: 3.27 s, total: 7min 36s
Wall time: 7min 39s


In [23]:
p = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(p, 'r_lda.html')